In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
basepath="/content/gdrive/MyDrive/Glaicer_Pytorch/"
import sys

sys.path.append(basepath)

In [3]:
# !git clone https://github.com/storyinvisible/Glaicer_Pytorch.git /content/gdrive/MyDrive/Glaicer_Pytorch/
!cd /content/gdrive/MyDrive/Glaicer_Pytorch/ && git pull
!mkdir plots
!pip install netCDF4~=1.5.6

In [4]:
import os.path
import torch
from torch.utils.data import DataLoader
from runner2D import trainer
from datasets import Glacier_dmdt, ERA5Datasets, GlacierDataset
from models import GlacierModel, LSTMPredictor, VCNN, HCNN

cuda = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_function = torch.nn.MSELoss()
save_path = "/content/gdrive/MyDrive/Glaicer_Pytorch/saved_models"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [5]:
epochs = 1
save_every = 1
eval_every = 1
hidden = [256, 128, 64, 32]

# STORSTROMMEN 2D
## STORSTROMMEN 2D Data

In [6]:
glacier_name = "STORSTROMMEN"
start, mid, end = 1979, 2008, 2018
train_smb = Glacier_dmdt(glacier_name, start, mid, path=os.path.join(basepath, "glacier_dmdt.csv"))
train_data = ERA5Datasets(glacier_name, start, mid, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
test_smb = Glacier_dmdt(glacier_name, start, end, path=os.path.join(basepath, "glacier_dmdt.csv"))
test_data = ERA5Datasets(glacier_name, start, end, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
glacier_dataset = GlacierDataset([train_data], [train_smb])
loader = DataLoader(glacier_dataset, batch_size=16, shuffle=False)

## STORSTROMMEN 2D HCNN

In [7]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = HCNN(in_channel=5, output_dim=256, vertical_dim=326)
glacier_model = GlacierModel(extractor, lstm_model, name="HCNNLSTM-STOR")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 5.1127 Eval: 0.8525
[INFO] Epoch 1|1 2 Loss: 5.1131 Eval: 0.8356


## STORSTROMMEN 2D VCNN

In [8]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = VCNN(in_channel=5, output_dim=256)
glacier_model = GlacierModel(extractor, lstm_model, name="VCNNLSTM-STOR")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 5.0755 Eval: 0.8441
[INFO] Epoch 1|1 2 Loss: 5.0760 Eval: 0.8389


# HELHEIMGLETSCHER 2D
## HELHEIMGLETSCHER 2D Data

In [9]:
glacier_name = "HELHEIMGLETSCHER"
start, mid, end = 1980, 2008, 2018
train_smb = Glacier_dmdt(glacier_name, start, mid, path=os.path.join(basepath, "glacier_dmdt.csv"))
train_data = ERA5Datasets(glacier_name, start, mid, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
test_smb = Glacier_dmdt(glacier_name, start, end, path=os.path.join(basepath, "glacier_dmdt.csv"))
test_data = ERA5Datasets(glacier_name, start, end, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
glacier_dataset = GlacierDataset([train_data], [train_smb])
loader = DataLoader(glacier_dataset, batch_size=16, shuffle=False)

## HELHEIMGLETSCHER HCNN

In [10]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = HCNN(in_channel=5, output_dim=256, vertical_dim=167)
glacier_model = GlacierModel(extractor, lstm_model, name="HCNNLSTM-HELH")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 33.1497 Eval: 18.7463
[INFO] Epoch 1|1 2 Loss: 84.5874 Eval: 18.6182


## HELHEIMGLETSCHER VCNN

In [11]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = VCNN(in_channel=5, output_dim=256)
glacier_model = GlacierModel(extractor, lstm_model, name="VCNNLSTM-HELH")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 33.5711 Eval: 18.9303
[INFO] Epoch 1|1 2 Loss: 85.4744 Eval: 18.7234


# JAKOBSHAVN 2D
## JAKOBSHAVN 2D Data

In [12]:
glacier_name = "JAKOBSHAVN_ISBRAE"
start, mid, end = 1980, 2008, 2018
train_smb = Glacier_dmdt(glacier_name, start, mid, path=os.path.join(basepath, "glacier_dmdt.csv"))
train_data = ERA5Datasets(glacier_name, start, mid, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
test_smb = Glacier_dmdt(glacier_name, start, end, path=os.path.join(basepath, "glacier_dmdt.csv"))
test_data = ERA5Datasets(glacier_name, start, end, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
glacier_dataset = GlacierDataset([train_data], [train_smb])
loader = DataLoader(glacier_dataset, batch_size=16, shuffle=False)

## JAKOBSHAVN HCNN

In [13]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = HCNN(in_channel=5, output_dim=256, vertical_dim=289)
glacier_model = GlacierModel(extractor, lstm_model, name="HCNNLSTM-JAKOB")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 41.6563 Eval: 30.9484
[INFO] Epoch 1|1 2 Loss: 120.2730 Eval: 30.7945


## JAKOBSHAVN VCNN

In [14]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = VCNN(in_channel=5, output_dim=256)
glacier_model = GlacierModel(extractor, lstm_model, name="VCNNLSTM-JAKOB")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 41.0993 Eval: 30.7634
[INFO] Epoch 1|1 2 Loss: 119.2609 Eval: 30.5573


# DAUGAARD-JENSEN 2D
## DAUGAARD-JENSEN 2D Data

In [15]:
glacier_name = "DAUGAARD-JENSEN"
start, mid, end = 1987, 2010, 2018
train_smb = Glacier_dmdt(glacier_name, start, mid, path=os.path.join(basepath, "glacier_dmdt.csv"))
train_data = ERA5Datasets(glacier_name, start, mid, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
test_smb = Glacier_dmdt(glacier_name, start, end, path=os.path.join(basepath, "glacier_dmdt.csv"))
test_data = ERA5Datasets(glacier_name, start, end, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
glacier_dataset = GlacierDataset([train_data], [train_smb])
loader = DataLoader(glacier_dataset, batch_size=16, shuffle=False)

## DAUGAARD-JENSEN VCNN

In [16]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = VCNN(in_channel=5, output_dim=256)
glacier_model = GlacierModel(extractor, lstm_model, name="VCNNLSTM-DAUG")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 4.9549 Eval: 2.5298
[INFO] Epoch 1|1 2 Loss: 9.6970 Eval: 2.4732


## DAUGAARD-JENSEN HCNN

In [17]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = HCNN(in_channel=5, output_dim=256, vertical_dim=210)
glacier_model = GlacierModel(extractor, lstm_model, name="HCNNLSTM-DAUG")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 5.1170 Eval: 2.6078
[INFO] Epoch 1|1 2 Loss: 9.9970 Eval: 2.5461


# QAJUUTTAP 2D
## QAJUUTTAP 2D Data

In [18]:
glacier_name = "QAJUUTTAP_SERMIA"
start, mid, end = 1987, 2010, 2018
train_smb = Glacier_dmdt(glacier_name, start, mid, path=os.path.join(basepath, "glacier_dmdt.csv"))
train_data = ERA5Datasets(glacier_name, start, mid, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
test_smb = Glacier_dmdt(glacier_name, start, end, path=os.path.join(basepath, "glacier_dmdt.csv"))
test_data = ERA5Datasets(glacier_name, start, end, path=os.path.join(basepath, "ECMWF_reanalysis_data"))
glacier_dataset = GlacierDataset([train_data], [train_smb])
loader = DataLoader(glacier_dataset, batch_size=16, shuffle=False)

## QAJUUTTAP VCNN

In [19]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = VCNN(in_channel=5, output_dim=256)
glacier_model = GlacierModel(extractor, lstm_model, name="VCNNLSTM-QAJU")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 1.2388 Eval: 0.6002
[INFO] Epoch 1|1 2 Loss: 2.1799 Eval: 0.5648


## QAJUUTTAP HCNN

In [20]:
lstm_model = LSTMPredictor(layers=None, input_dim=256, hidden_dim=hidden, n_layers=1, bidirection=False, p=0.5)
extractor = HCNN(in_channel=5, output_dim=256, vertical_dim=14)
glacier_model = GlacierModel(extractor, lstm_model, name="HCNNLSTM-QAJU")
trainer(glacier_model, train_loader=loader, testdataset=test_data, testsmb=test_smb,
        show=False, device=cuda, epochs=epochs, lr=0.002, reg=0.001, save_every=save_every, eval_every=eval_every, test_split_at=mid,
        critic=loss_function, optimizer=torch.optim.Adam, save_path=save_path)

[INFO] Epoch 1|1 1 Loss: 1.2767 Eval: 0.6284
[INFO] Epoch 1|1 2 Loss: 2.2648 Eval: 0.5954
